In [ ]:
# eye segmentation using YOLOv8
import sys
import subprocess

# function to install a package
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# install necessary packages if not already installed
def install_packages():
    packages = ['ultralytics', 'opencv-python-headless', 'Pillow', 'scikit-learn', 'numpy']
    for package in packages:
        try:
            __import__(package.split('-')[0])  # import the base package name
        except ImportError:
            print(f"Installing {package}...")
            install(package)

install_packages()

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import cv2
from ultralytics import YOLO
from albumentations import Compose, HorizontalFlip, RandomCrop, Resize

In [ ]:
# define paths
images_path = '/content/drive/MyDrive/dataset/images/Images_left_eye'
masks_path = '/content/drive/MyDrive/dataset/masks/left_eye_segmask/SegmentationClass'

# paths for train and validation sets
train_images_path = '/content/drive/MyDrive/dataset/train/images'
train_masks_path = '/content/drive/MyDrive/dataset/train/mask'
val_images_path = '/content/drive/MyDrive/dataset/val/images'
val_masks_path = '/content/drive/MyDrive/dataset/val/mask'

# create directories if they don't exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_masks_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_masks_path, exist_ok=True)

In [ ]:
# list all image files and filter those with corresponding masks
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg') or f.endswith('.png')]

images_with_masks = []
for file in image_files:
    filename_base = os.path.splitext(file)[0]
    mask_file = os.path.join(masks_path, filename_base + '.png')  # assuming masks are .png files
    if os.path.exists(mask_file):
        images_with_masks.append(file)
    else:
        print(f"No mask found for {file}; excluding from dataset.")

In [ ]:
# resize images and masks to a fixed size and apply augmentations
def preprocess_image_and_mask(image_path, mask_path, output_image_path, output_mask_path):
    # load image and mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if image is None or mask is None:
        print(f"Failed to load {image_path} or {mask_path}. Skipping.")
        return

    # resize both image and mask to 640x640
    resize_transform = Resize(height=640, width=640, interpolation=cv2.INTER_NEAREST)
    augmented = resize_transform(image=image, mask=mask)
    resized_image = augmented['image']
    resized_mask = augmented['mask']

    # save resized image and mask
    cv2.imwrite(output_image_path, resized_image)
    cv2.imwrite(output_mask_path, resized_mask)

def augment_data(image_path, mask_path, output_image_path, output_mask_path):
    # define augmentations
    augmentations = Compose([
        HorizontalFlip(p=0.5),
        RandomCrop(height=512, width=512, p=1.0)
    ])

    # load image and mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if image is None or mask is None:
        print(f"Failed to load {image_path} or {mask_path}. Skipping.")
        return

    # apply augmentations
    augmented = augmentations(image=image, mask=mask)
    augmented_image = augmented['image']
    augmented_mask = augmented['mask']

    # save augmented data
    cv2.imwrite(output_image_path, augmented_image)
    cv2.imwrite(output_mask_path, augmented_mask)

In [ ]:
# preprocess and copy images and masks to train and val directories
for file in images_with_masks:
    filename_base = os.path.splitext(file)[0]
    image_path = os.path.join(images_path, file)
    mask_path = os.path.join(masks_path, filename_base + '.png')

    # split into train and val sets
train_files, val_files = train_test_split(images_with_masks, test_size=0.2, random_state=42)

# preprocess and copy images and masks to train and val directories
for file in images_with_masks:
    filename_base = os.path.splitext(file)[0]
    image_path = os.path.join(images_path, file)
    mask_path = os.path.join(masks_path, filename_base + '.png')

    if file in train_files:
        preprocess_image_and_mask(image_path, mask_path,
                                  os.path.join(train_images_path, file),
                                  os.path.join(train_masks_path, filename_base + '.png'))
    else:
        preprocess_image_and_mask(image_path, mask_path,
                                  os.path.join(val_images_path, file),
                                  os.path.join(val_masks_path, filename_base + '.png'))

In [ ]:
# convert masks to YOLO segmentation labels
def convert_masks_to_yolo_format(images_dir, masks_dir, labels_dir):
    os.makedirs(labels_dir, exist_ok=True)
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]

    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, os.path.splitext(image_file)[0] + '.png')
        label_path = os.path.join(labels_dir, os.path.splitext(image_file)[0] + '.txt')

        if not os.path.exists(mask_path):
            print(f"Mask not found for {image_file}, skipping.")
            continue

        # read image to get dimensions
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to read image {image_file}, skipping.")
            continue
        height, width = img.shape[:2]

        # read and process mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Failed to read mask {mask_path}, skipping.")
            continue

        # ensure binary mask
        mask = np.where(mask > 0, 1, 0).astype(np.uint8)

        # find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            print(f"No contours found for {image_file}, skipping.")
            continue

        # prepare label file content
        label_lines = []
        for contour in contours:
            if len(contour) < 3:
                continue  # not enough points to form a polygon

            # bounding box
            x, y, w, h = cv2.boundingRect(contour)

            # reduce bounding box dimensions by scaling down width and height
            x += int(w * 0.25)
            y += int(h * 0.25)
            w = int(w * 0.5)
            h = int(h * 0.5)
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            bbox_width = (w / width)*0.5
            bbox_height = (h / height)*0.5

            # segmentation polygon
            contour = contour.flatten()
            if len(contour) < 6:
                continue  # skip small contours

            segmentation = []
            for i in range(0, len(contour), 2):
                x_point = contour[i] / width
                y_point = contour[i + 1] / height
                segmentation.append(f"{x_point:.6f}")
                segmentation.append(f"{y_point:.6f}")
            segmentation_str = ' '.join(segmentation)

            # class ID (assuming 'eye' class is 0)
            class_id = 0

            # combine into label line
            label_line = f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f} {segmentation_str}"
            label_lines.append(label_line)

        if not label_lines:
            print(f"No valid segmentation found for {image_file}, skipping.")
            continue

        # Write to label file
        with open(label_path, 'w') as f:
            f.write('\n'.join(label_lines))

        print(f"Processed {image_file}")

# paths to labels directories
train_labels_path = '/content/drive/MyDrive/dataset/train/labels'
val_labels_path = '/content/drive/MyDrive/dataset/val/labels'

# convert training masks
convert_masks_to_yolo_format(train_images_path, train_masks_path, train_labels_path)

# convert validation masks
convert_masks_to_yolo_format(val_images_path, val_masks_path, val_labels_path)

In [ ]:
# create data.yaml file
data_yaml_content = f"""
path: /content/drive/MyDrive/dataset

train: train/images
val: val/images

# Number of classes
nc: 3


# Class names
names: ['eye', 'colorcard', 'bluecircle']
"""

data_yaml_path = '/content/drive/MyDrive/dataset/data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

# display the data.yaml content
print("\nCreated data.yaml file:")
with open(data_yaml_path, 'r') as f:
    print(f.read())


In [ ]:
# train the YOLOv8 segmentation model
# load the YOLOv8 segmentation model
model = YOLO('yolov8x-seg.pt')

# Train the model
model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    name='eye_segmentation',
    task='segment',
    save = True
)

In [ ]:
model.val(data='/content/drive/MyDrive/dataset/data.yaml', imgsz=1024)

In [ ]:
results = model.predict(source='/content/drive/MyDrive/dataset/test_images/1709617535926.jpg', save=True, imgsz=640)

In [ ]:
from IPython.display import Image
display(Image(filename='runs/segment/train3/1709617535926.jpg'))

In [ ]:
# run inference on test images
test_images_path = '/content/drive/MyDrive/dataset/test_images'
output_folder = '/content/drive/MyDrive/dataset/Predictions241201_x'

# create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# run inference on all images in the test images folder
results = model.predict(
    source=test_images_path,
    save=True,
    conf=0.65,
    imgsz=640,
    task='segment'
)

# move the predicted images to the output folder
for file_name in os.listdir(model.predictor.save_dir):
    if file_name.endswith(('.jpg', '.jpeg', '.png')):
        src_path = os.path.join(model.predictor.save_dir, file_name)
        dst_path = os.path.join(output_folder, file_name)
        shutil.move(src_path, dst_path)

print(f"Predictions saved to: {output_folder}")